In [49]:
%%script false --no-raise-error
#Copy this code and run it in the terminal everytime you create a new directory or file
#Gives permission for everyone to work with the files created
#Run at the end of a jupyter script, the output is too messy
!cd /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_Bjork 
!chmod -R 777 . 

# Create transcript library 
1. Download human genome from ensembl: https://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
2. Create transcript fasta with bedtools getfasta.

In [11]:
%%bash
cd /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_Bjork
TRANSCRIPT_FILE='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.cdna.all.fa'
GTF_ANNOTATION='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.gtf'

# 1. 
# -nc makes sure that if the file is already there, it won't be overwritten.
# -nc can be changed to -N, if it is of interest to import newer versions
# -P will save it to the directory specified, in this case data/cDNA_library
# wget -nc https://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -P data/homo_sapiens_ensembl
# gzip -d data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
# rm data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

# 2.
# bedtools getfasta \
# -fi $GENOME_FILE \
# -bed $GTF_ANNOTATION \
# -fo data/homo_sapiens_ensembl/transcriptome_assembly.fa
# remove unzipped genome to save server space
# rm data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


# Build Bowtie index
This takes around 30 minutes to run on one kernel, so don't run it if you don't have to. For the bedtools index: started ~18:12 on two kernels, ended before 18:24

In [ ]:
%%bash
TRANSCRIPT_FILE='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.cdna.all.fa'

echo "Transcript file: $TRANSCRIPT_FILE"
echo "Started..."
rm -rf data/Bowtie_index
mkdir data/Bowtie_index

bowtie2-build \
-f $TRANSCRIPT_FILE \
data/Bowtie_index/hg38_cdna_index \
--threads 8
echo "Done!"
bowtie2-inspect -s data/Bowtie_index/hg38_cdna_index

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Transcript file: data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.cdna.all.fa
Started...
Settings:
  Output files: "data/Bowtie_index/hg38_cdna_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 32
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.cdna.all.fa


Building a SMALL index


Reading reference sizes
  Time reading reference sizes: 00:00:06
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:02
bmax according to bmaxDivN setting: 12344445
Using parameters --bmax 9258334 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9258334 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:06
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:03
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:03
  Sanity-checking and returning
Building samples
Reserving space for 86 sample suffixes
Generating random suffixes
QSorting 86 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey Q

# Run Bowtie2
Started ~14:00, ended ~14:50 with one core.

Started ~15:00, ended ~15:20 with two cores

In [ ]:
%%bash

mkdir -p data/alignment_Bowtie2_subset

# Loop over all FASTQ files ending in _1.fastq.gz in the subset_trimmed_RNA-seq folder
for R1 in data/subset_trimmed_RNA-seq/*_1.fastq.gz; do
  
  # Extract the base name 
  BN=$(basename "$R1")
  
  # Remove the _1.fastq.gz suffix to get the sample prefix 
  SAMPLE=${BN%%_1.fastq.gz}
  
  # Construct the path to the mate 2 file 
  R2="data/subset_trimmed_RNA-seq/${SAMPLE}_2.fastq.gz"
  
  echo "Aligning $SAMPLE with Bowtie2..."
  
  # Run Bowtie2 in local alignment mode for the paired-end reads
  bowtie2 --local \
    -x data/Bowtie_index/hg38_cdna_index \
    -1 "$R1" -2 "$R2" \
    -S data/alignment_Bowtie2_subset/${SAMPLE}.sam \
    --threads 4
  
  # Convert the SAM to a sorted BAM and create an index using samtools
  samtools view -bS data/alignment_Bowtie2_subset/${SAMPLE}.sam | samtools sort -o data/alignment_Bowtie2_subset/${SAMPLE}.bam
  samtools index data/alignment_Bowtie2_subset/${SAMPLE}.bam
  
  # Remove the SAM file to save disk space
  rm data/alignment_Bowtie2_subset/${SAMPLE}.sam
  
  echo "Done aligning $SAMPLE"
  echo "-----------------------------------------"
done


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Aligning ERR950158_trimmed with Bowtie2...


499532 reads; of these:
  499532 (100.00%) were paired; of these:
    261310 (52.31%) aligned concordantly 0 times
    62637 (12.54%) aligned concordantly exactly 1 time
    175585 (35.15%) aligned concordantly >1 times
    ----
    261310 pairs aligned concordantly 0 times; of these:
      1885 (0.72%) aligned discordantly 1 time
    ----
    259425 pairs aligned 0 times concordantly or discordantly; of these:
      518850 mates make up the pairs; of these:
        486970 (93.86%) aligned 0 times
        6480 (1.25%) aligned exactly 1 time
        25400 (4.90%) aligned >1 times
51.26% overall alignment rate


Done aligning ERR950158_trimmed
-----------------------------------------
Aligning ERR950159_trimmed with Bowtie2...


499704 reads; of these:
  499704 (100.00%) were paired; of these:
    303889 (60.81%) aligned concordantly 0 times
    67440 (13.50%) aligned concordantly exactly 1 time
    128375 (25.69%) aligned concordantly >1 times
    ----
    303889 pairs aligned concordantly 0 times; of these:
      5498 (1.81%) aligned discordantly 1 time
    ----
    298391 pairs aligned 0 times concordantly or discordantly; of these:
      596782 mates make up the pairs; of these:
        562995 (94.34%) aligned 0 times
        7124 (1.19%) aligned exactly 1 time
        26663 (4.47%) aligned >1 times
43.67% overall alignment rate


...

Done aligning ERR950181_trimmed
-----------------------------------------
Aligning ERR950182_trimmed with Bowtie2...


499213 reads; of these:
  499213 (100.00%) were paired; of these:
    257178 (51.52%) aligned concordantly 0 times
    74321 (14.89%) aligned concordantly exactly 1 time
    167714 (33.60%) aligned concordantly >1 times
    ----
    257178 pairs aligned concordantly 0 times; of these:
      2838 (1.10%) aligned discordantly 1 time
    ----
    254340 pairs aligned 0 times concordantly or discordantly; of these:
      508680 mates make up the pairs; of these:
        473492 (93.08%) aligned 0 times
        8413 (1.65%) aligned exactly 1 time
        26775 (5.26%) aligned >1 times
52.58% overall alignment rate


Done aligning ERR950182_trimmed
-----------------------------------------
Aligning ERR950186_trimmed with Bowtie2...


498509 reads; of these:
  498509 (100.00%) were paired; of these:
    249406 (50.03%) aligned concordantly 0 times
    78249 (15.70%) aligned concordantly exactly 1 time
    170854 (34.27%) aligned concordantly >1 times
    ----
    249406 pairs aligned concordantly 0 times; of these:
      5881 (2.36%) aligned discordantly 1 time
    ----
    243525 pairs aligned 0 times concordantly or discordantly; of these:
      487050 mates make up the pairs; of these:
        454211 (93.26%) aligned 0 times
        6884 (1.41%) aligned exactly 1 time
        25955 (5.33%) aligned >1 times
54.44% overall alignment rate


Done aligning ERR950186_trimmed
-----------------------------------------
Aligning ERR950187_trimmed with Bowtie2...


In [3]:
%%bash
samtools flagstat data/alignment_Bowtie2_subset/ERR950187_trimmed.bam

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


998450 + 0 in total (QC-passed reads + QC-failed reads)
998450 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
450566 + 0 mapped (45.13% : N/A)
450566 + 0 primary mapped (45.13% : N/A)
998450 + 0 paired in sequencing
499225 + 0 read1
499225 + 0 read2
422530 + 0 properly paired (42.32% : N/A)
438398 + 0 with itself and mate mapped
12168 + 0 singletons (1.22% : N/A)
11068 + 0 with mate mapped to a different chr
4064 + 0 with mate mapped to a different chr (mapQ>=5)


# Download .gtf file for summarization
https://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens
/Homo_sapiens.GRCh38.113.gtf.gz

In [23]:
%%script false --no-raise-error
%%bash
# -nc makes sure that if the file is already there, it won't be overwritten.
# -nc can be changed to -N, if it is of interest to import newer versions
# -P will save it to the directory specified, in this case data/cDNA_library
wget -nc https://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens/Homo_sapiens.GRCh38.113.gtf.gz -P data/
gzip -d data/Homo_sapiens.GRCh38.113.gtf.gz
rm data/Homo_sapiens.GRCh38.113.gtf.gz

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
--2025-03-04 21:02:22--  https://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens/Homo_sapiens.GRCh38.113.gtf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64141785 (61M) [application/x-gzip]
Saving to: 'data/Homo_sapiens.GRCh38.113.gtf.gz'

     0K .......... .......... .......... .......... ..........  0% 1.06M 58s
    50K .......... .......... .......... .......... ..........  0% 2.07M 44s
   100K .......... .......... .......... .......... ..........  0%  112M 29s
   150K .......... .......... .......... .......... ..........  0%  162M 22s
   200K .......... .......... .......... .......... ..........  0% 2.09M 23s
   250K .......... .......... .......... .......... ..........  0%  142M 20s
   300K .......

CalledProcessError: Command 'b"# -nc makes sure that if the file is already there, it won't be overwritten.\n# -nc can be changed to -N, if it is of interest to import newer versions\n# -P will save it to the directory specified, in this case data/cDNA_library\nwget -nc https://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens/Homo_sapiens.GRCh38.113.gtf.gz -P data/\ngzip -d data/Homo_sapiens.GRCh38.113.gtf.gz\nrm data/Homo_sapiens.GRCh38.113.gtf.gz\n"' returned non-zero exit status 1.

# Summarize with featureCounts

In [ ]:
%%bash
GTF_ANNOTATION='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.gtf'

# path to the GTF annotation, output file, -p to specify paired-end reads 
# -C to remove pairs that have been mapped to different chromosomes 
# takes ~30 sek to run on two cores
awk '$3 == "transcript"' 'data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.gtf' > data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.transcripts.gtf

featureCounts -T 8 -t transcript -g transcript_id \
  -a data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.transcripts.gtf \
  -o data/Bowtie2_gene_counts/bowtie_featureCounts.txt \
  -p data/alignment_Bowtie2_subset/*.bam

# --countReadPairs
# -A data/featureCounts_alias/featureCounts_alias.txt \ to include chromosome aliases

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory

        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.8

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 14 BAM files                                     ||
||                                                                            ||
||                           ERR950158_trimmed.bam                            ||
||                   

# Get gene counts with HTSeq
Because featureCounts is obviously not working.

In [2]:
%%script false --no-raise-error
%%bash
htseq-count -f bam -t exon data/alignment_Bowtie2_subset/*.bam '$GTF_ANNOTATION'


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
  [Errno 2] No such file or directory: '$GTF_ANNOTATION'
  [Exception type: FileNotFoundError, raised in utils.py:48]


CalledProcessError: Command 'b"htseq-count -f bam -t exon data/alignment_Bowtie2_subset/*.bam '$GTF_ANNOTATION'\n"' returned non-zero exit status 1.

# Remember to change access for everyone

In [ ]:
%%bash
chmod -R 777 . 

In [ ]:
! 